In [ ]:
"""
Produce LandScape File
"""

con_db = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'fireloc_db'
}

db_schema = {
    "GRID" : 'grid_ref', "GRID_PK" : 'gid', 'CELLID' : 'cellid',
    "DATA" : 'datasets', 'DATA_PK' : 'did', 'SLUG' : 'slug',
    "REL"  : 'rel_grid_data', 'GRID_FK' : 'gid', "DATA_FK" : 'did',
    "SHP"  : 'shp', 'RST' : 'rst'
}

main_folder = '/home/jasp/datasets'

epsg = 3763

lmt = '/home/jasp/tst_farsite/lmt_105.shp'

cos_slug = 'cos15_lvl3'
dem_slug = 'dem888_m50'
tcd_slug = 'tcd_15'

climatic_data = '/home/jasp/tst_farsite/data_meteo.xlsx'
climatic_sheet = 'coimbra_wsx'
climatic_elevation = 171

procedure = '/home/jasp/tst_farsite/'

import os; import codecs
from osgeo import gdal
import numpy as np
from gasp.gt.prop.ext import get_ext
from gasp.g.to import shpext_to_boundary
from gasp.sql.fm import Q_to_df
from gasp.gt.nop.mos import rsts_to_mosaic
from gasp.gt.nop.rcls import rcls_rst
from gasp.gt.wenv.grs import run_grass
from gasp.fm import tbl_to_obj
from gasp.gt.nop.alg import floatrst_to_intrst
from gasp.g.prop.img import get_cell_size, get_nd
from gasp.gt.to.rst import obj_to_rst

In [ ]:
"""
Generate file with climatic data
"""

meteo_df = tbl_to_obj(climatic_data, sheet=climatic_sheet)

txt_data = (
    'RAWS_ELEVATION: {}\n'
    'RAWS_WINDS: Ave\n'
    'RAWS_UNITS: Metric\n\n'
    'Year  Mth  Day   Time    Temp     RH    HrlyPcp   WindSpd  WindDir CloudCov\n{}'
)
__txt = ''

for idx, row in meteo_df.iterrows():
    __txt += (
        '{year}{a}{month}{b}{day}{c}{tim}{d}{temp}{e}{rh}{f}'
        '{prec}.00{g}{wspeed}{h}{wdir}{i}{cloud}\n'
    ).format(
        year=str(row.Year), a=' ' * 3,
        month=str(row.Mth) if len(str(row.Mth)) == 2 else ' ' + str(row.Mth),
        b=' ' * 3,
        day=str(row.Day) if len(str(row.Day)) == 2 else ' ' + str(row.Day),
        c=' ' * 3, tim=str(row.Time), d=' ' * 6,
        temp=str(row.Temp) if len(str(row.Temp)) == 2 else ' ' + str(row.Temp),
        e=' ' * 5, rh=str(row.RH) if len(str(row.RH)) == 2 else ' ' + str(row.RH),
        f=' ' * 4, prec=str(row.HrlyPcp) if len(str(row.HrlyPcp)) == 4 else \
            ' ' * (4 -  len(str(row.HrlyPcp))) + str(row.HrlyPcp),
        g=' ' * 6, wspeed=str(row.WindSpd) if len(str(row.WindSpd)) == 3 else \
            ' ' * (3 - len(str(row.WindSpd))) + str(row.WindSpd),
        h=' ' * 5, wdir=str(row.WindDir) if len(str(row.WindDir)) == 3 else \
            ' ' * (3 - len(str(row.WindDir))) + str(row.WindDir),
        i=' ' * 4, cloud=str(row.CloudCov) if len(str(row.CloudCov)) == 2 else ' ' + str(row.CloudCov)
    )

with codecs.open(os.path.join(procedure, 'climatic_txt.wxs'), 'w', encoding='utf-8') as txt:
    txt.write(txt_data.format(str(climatic_elevation), __txt))

In [ ]:
lmt_geom = shpext_to_boundary(lmt).ExportToWkt()

Q = (
    "SELECT foo.{gid}, foo.{cellid}, jtbl.{slug_col}, "
    "jtbl.{rst}, jtbl.{shp} "
    "FROM ("
        "SELECT *, "
        "(ST_Area(ST_Intersection(geom, ST_GeomFromText('{wkt}', {srs}))) / ST_Area(geom) * 100) AS i_area "
        "FROM {gridt} "
        "WHERE ST_Intersects(geom, ST_GeomFromText('{wkt}', {srs}))"
    ") AS foo INNER JOIN ("
        "SELECT mtbl.{did}, mtbl.{slug_col}, "
        "jtbl.{relgid}, jtbl.{rst}, jtbl.{shp} "
        "FROM {datat} AS mtbl "
        "INNER JOIN {reltbl} AS jtbl ON "
        "mtbl.{did} = jtbl.{reldid} "
        "WHERE {slug_col}='{slug}' OR {slug_col}='{slug_dem}' "
        "OR {slug_col}='{slug_tcd}'"
    ") AS jtbl ON foo.{gid} = jtbl.{relgid} "
    "WHERE i_area > 1"
).format(
    gid=db_schema['GRID_PK'], cellid=db_schema['CELLID'],
    rst=db_schema['RST'], shp=db_schema['SHP'],
    wkt=lmt_geom, srs=str(epsg), gridt=db_schema["GRID"],
    datat=db_schema["DATA"],
    relgid=db_schema["GRID_FK"], reltbl=db_schema["REL"],
    did=db_schema["DATA_PK"], reldid=db_schema["DATA_FK"],
    slug=cos_slug, slug_col=db_schema["SLUG"],
    slug_dem=dem_slug, slug_tcd=tcd_slug
)

cells_df = Q_to_df(con_db, Q)
cells_df['rst'] = main_folder + '/' + cells_df.rst

In [ ]:
"""
Get DEM
"""

# Get DEM
dem_rsts = cells_df[cells_df[db_schema["SLUG"]] == dem_slug].rst.tolist()

dem_rst = dem_rsts[0] if len(dem_rsts) == 1 else rsts_to_mosaic(
    dem_rsts, os.path.join(procedure, 'dem_rst.tif'), api='rasterio')

# Open DEM Raster
dem_src = gdal.Open(dem_rst)

# Get DEM Cellsize
dem_cs = int(get_cell_size(dem_src)[0])

In [ ]:
def rst_generalize(refrst, valrst, refcs, valcs, out_rst):
    """
    Get Majority value for each cell in ref raster
    
    Useful when ref raster has cellsize major than
    value raster
    """
    
    # Data to Array
    refnum = refrst.ReadAsArray()
    valnum = valrst.ReadAsArray()
    
    # Get Ref shape
    ref_shape = refnum.shape
    
    # in a row, how many cells valnum are for each refnum cell
    dcell = int(refcs / valcs)
    
    # Valnum must be of type int
    
    # Create generalized raster
    resnum = np.zeros(ref_shape, dtype=valnum.dtype)
    
    for row in range(ref_shape[0]):
        for col in range(ref_shape[1]):
            resnum[row, col] = np.bincount(
                valnum[row*dcell:row*dcell+dcell, col*dcell:col*dcell+dcell].reshape(dcell*dcell)
            ).argmax()
    
    # Export out raster
    return obj_to_rst(resnum, out_rst, refrst, noData=get_nd(valrst))

In [ ]:
# Get COS_rst
cos_rst = cells_df[cells_df[db_schema["SLUG"]] == cos_slug].rst.tolist()

lulc_rst = cos_rst[0] if len(cos_rst) == 1 else rsts_to_mosaic(
    cos_rst, os.path.join(procedure, 'mos_lulc.tif'), api='rasterio'
)

# Check if COS RST has the same cellsize than dem
lulc_src = gdal.Open(lulc_rst)
lulc_cs = int(get_cell_size(lulc_src)[0])

if lulc_cs == dem_cs:
    lulc_rst = lulc_rst
elif lulc_cs < dem_cs:
    lulc_rst = rst_generalize(dem_src, lulc_src, dem_cs, lulc_cs, os.path.join(procedure, 'gen_lulc.tif'))
else:
    raise ValueError('script doesn\'t know what to do...')

In [ ]:
# Reclass COS
# CLS to Fuel Model
rules = {
    (0, 11) : 90,
    (11, 19) : 1,
    20 : 5,
    21 : 1,
    22 : 9,
    23 : 6,
    24 : 1,
    25 : 6,
    26 : 8,
    27 : 90,
    (27, 32) : 98
}

rcls_lulc = rcls_rst(lulc_rst, rules, os.path.join(procedure, 'fuel_rst.tif'))

In [ ]:
# Get Tree Canocopy cover

tcd_rsts = cells_df[cells_df[db_schema["SLUG"]] == tcd_slug].rst.tolist()

tcd_rst = tcd_rsts[0] if len(tcd_rsts) == 1 else rsts_to_mosaic(
    tcd_rsts, os.path.join(procedure, 'tcd.tif'), api='rasterio')

# Check if COS RST has the same cellsize than dem
tcd_src = gdal.Open(tcd_rst)
tcd_cs = int(get_cell_size(tcd_src)[0])

if tcd_cs == dem_cs:
    tcd_rst = tcd_rst
elif tcd_cs < dem_cs:
    tcd_rst = rst_generalize(dem_src, tcd_src, dem_cs, tcd_cs, os.path.join(procedure, 'gen_tcd.tif'))
else:
    raise ValueError('script doesn\'t know what to do...')

# Reclass
rules = {0 : 99, (0, 20) : 1, (20, 50) : 2, (50, 80) : 3, (80, 100) : 4}

tcc_rst = rcls_rst(tcd_rst, rules, os.path.join(procedure, 'tcc.tif'))

In [ ]:
# Create Slope and Aspect Raster
grsbase = run_grass(procedure, grassBIN='grass78', location='dem_asp', srs=epsg)

import grass.script as grass
import grass.script.setup as gsetup
gsetup.init(grsbase, procedure, 'dem_asp', 'PERMANENT')

from gasp.gt.to.rst import rst_to_grs, grs_to_rst
from gasp.gt.wenv.grs import rst_to_region
from gasp.gt.nop.surf import slope, aspect

# Configure region
rst_ext = rst_to_grs(dem_rst, 'dem')
rst_to_region(rst_ext)

# Produce slope raster
dclv = slope(rst_ext, 'slope_rst', api='grass')

# Produce aspect raster
expo = aspect(rst_ext, 'aspect_rst', api='grass', from_north=True)

# Export data
dclv = grs_to_rst(dclv, os.path.join(procedure, 'dclv.tif'))
expo = grs_to_rst(expo, os.path.join(procedure, 'aspect.tif'))

In [ ]:
"""
Convert Float Rasters to Integers Rasters
"""

for r in [dem_rst, dclv, expo]:
    floatrst_to_intrst(r, os.path.join(procedure, 'int_{}'.format(os.path.basename(r))))